# __WSI - ćwiczenie 1.__
### __Zagadnienie przeszukiwania i podstawowe podejścia do niego__


1. Narysować funkcje f(x) i g(x).
2. Zaimplementować algorytm najszybszego spadku oraz zastosować go do znalezienia minimum
funkcji f i g.
3. Zbadać wpływ rozmiaru kroku dla różnych (losowych) punktów początkowych.

In [52]:
import numpy as np
from plotly import graph_objs as go
import pandas as pd

RNG = np.random.default_rng()

#### __Definicje funkcji oraz ich gradientów__

In [53]:
def f(vect):
    assert vect.shape == (1,)
    return 10*vect[0]**4 + 3*vect[0]**3 - 30*vect[0]**2 + 10*vect[0]
    
def g(vect):
    assert vect.shape == (2,)
    return 10*vect[1]**4 + 10*vect[0]**4 + 3*vect[0]**3 - 30*vect[0]**2 + 10*vect[0]

def grad_f(vect):
    assert vect.shape == (1,)
    return np.array(40*vect[0]**3 + 9*vect[0]**2 - 60*vect[0] + 10)

def grad_g(vect):
    assert vect.shape == (2,)
    return np.array([40*vect[0]**3 + 9*vect[0]**2 - 60*vect[0] + 10,
                     40*vect[1]**3])

### 

#### __Algorytm najszybszego spadku__


In [54]:
def gradient_descent(start_point, beta, grad, stop_treshlod, num_iters):
    """
    Implementation of gradient descent for minimalisation purposes,
    usable with n-dimensional loss functions, if a suitable gradient 
    function is provided.

    Args:
        start_point: coordinates of the first point
        beta: learning rate of the algorithm = step coefficient
        grad: a function which returns a matrix of partial derivatives of the input function
        stop_treshlod: minimal value of gradient
        num_iters: maximal number of iterations (steps)

    Returns:
        Returns a tuple of an array of points generated by alogrithm
        (dependiong on the gradiant function the point format may vary),
        and a boolean value which depicts whether the function has succesfully
        performed all of its iterations or reached given stop treshold. 

    Raises:
        None
    """
    steps = np.array([start_point])
    point = start_point
    for _ in range(num_iters):
        theta = grad(point)
        # check if algorithm reached a local minimum,
        # i.e. func has flattened out
        # Could be changed for Euclidean distance, 
        # but this should perform faster 
        if abs(theta.sum()) < stop_treshlod:
            break
        # prevent overflow errors
        if abs(theta.sum()) > 100000:
            return steps, False
        # perform algorithm step
        point = point - beta * theta
        # add point to the output array
        steps = np.append(steps, [point], 0)
    return steps, True


#### __Generowanie wykresów__


Wykres dla funkcji jednej zmiennej f(x):

In [55]:
max_r = 3
X = np.linspace(-max_r, max_r, 100)
Y = np.array([f(np.array([x])) for x in X])

pt = RNG.uniform(-max_r, max_r, 1)
dsc = gradient_descent(pt, 0.005, grad_f, 0.01, 100)
if not dsc[1]:
    print("Algorithm hasn't found the optimum, steps are out of bounds")
else:
    steps = np.array([x for x in dsc[0]
                    if abs(x[0]) < max_r])
    XS = steps[:, 0]
    YS = np.array([f(x) for x in steps])

    layout = go.Layout(width=700, height=500,
                       title='Gradient Descent of single variable function',
                       xaxis_title='x',
                       yaxis_title='f(x)',
                       plot_bgcolor='DarkSeaGreen')
    fig = go.Figure(data=[go.Scatter(x=X, y=Y, line=dict(color='DarkSlateGrey', width=3))], 
                    layout=layout)
    fig.add_trace(go.Scatter(x=XS, y=YS, mode='lines+markers', 
                            marker=dict(size=6, color=YS,
                            colorscale='Agsunset'),
                            line=dict(color='DarkSlateGrey', width=1)))
    fig.show()

Wykres dla funkcji dwóch zmiennych g(x, y):

In [56]:
max_r = 3
l = np.linspace(-max_r, max_r, 100)
X, Y = l, l
Z = np.array([[g(np.array([x, y])) for x in X] for y in Y])

pt = RNG.uniform(-max_r, max_r, 2)
dsc = gradient_descent(pt, 0.005, grad_g, 0.1, 100)
if not dsc[1]:
    print("Algorithm hasn't found the optimum, steps are out of bounds")
else:
    steps = np.array([pt for pt in dsc[0] if abs(pt[0]) < max_r and abs(pt[1]) < max_r])
    XS, YS = steps[:,0], steps[:,1]
    ZS = np.array([g(np.array([x, y])) for x, y in zip(XS, YS)])

    layout = go.Layout(width = 700, height =700,
                       title_text='Gradient Descent minimalisation of double variable function',
                       xaxis_title='x',
                       yaxis_title='y')                       
    fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z, colorscale='Emrld',
                                    opacity=0.5)], layout=layout)
    fig.update_traces(contours_z=dict(show=True, usecolormap=True))
    fig.add_scatter3d(x=XS, y=YS, z=ZS, mode='lines+markers', 
                      marker=dict(size=4, color=ZS,               
                                  colorscale='Agsunset'),
                      line=dict(color='DarkSlateGrey', width=0.7))
    fig.update_layout(scene=dict(zaxis_title="g(x, y)"))
    fig.show()

#### __Analiza wydajności funkcji najszybszego spadku w zależności od wartości współczynnika kroku__

Z przeprowadzonych obserwacji wywnioskowałem, że potrzebuję co najmniej kilkuset iteracji (uruchomień dla punktów początkowych) by zbadać generalne zachowanie algorytmu. Badania przeprowadzałem głównie na obszarze od -3 do 3, na którym 100 kroków algorytmu było zdecydowanie wystarczające żeby funkcja zatrzymała się po osiągnięciu dolnego limitu wartości gradientu, oznaczonego współczynnikiem odcięcia.


In [57]:
vals = [g(p) for p in dsc[0]]
layout = go.Layout(width=700, height=500,
                title_text='g(x,y) function value for GDA steps',
                plot_bgcolor='DarkSeaGreen',
                xaxis_title='Traversed point number',
                yaxis_title='g(x, y)')
fig = go.Figure(layout=layout)
fig.add_trace(go.Scatter(mode='lines+markers', 
                         marker=dict(size=6,
                                     color='DarkSlateGrey'),
                         line=dict(color='DarkSlateGrey', width=1), 
                         x=list(range(len(vals))), y=vals))
fig.show()

Współczynnik odcięcia __stop_treshold__ ustawiłem na 0.01, co wydało mi się wystarczające żeby stwierdzić że funkcja wystarczajaco zbliżyła się do minimum lokalnego i się stamtąd nie ruszy.

Funkcja testująca współczynnik kroku __beta__ uruchamia algorytm gradientowy __num_iters__ razy dla losowych punktów w zakresie badanego obszzaru i zwraca listę punktów końcowych wywołań algorytmu.

In [58]:
def test_beta(func, func_parameter_num, grad, beta: int, max_r, num_iters: int):
    end_points = []
    for _ in range(num_iters):
        p = RNG.uniform(max_r[0], max_r[1], func_parameter_num)
        dsc = gradient_descent(p, beta, grad, 0.01, 100)
        if not dsc[1]:
            end_points.append(np.nan)
        else:
            end_points.append(func(dsc[0][-1]))
    return np.array(end_points)

Metodą inżynierską zbadałem, że optymalna wartość zmienia się w zależnosci od rozimaru powierzchni dopuszczalnej. Dlatego, oraz dlatego, że w przypadkach innych funkcji których wykresów nie znamy, korzystne będzie przebadać większy zakres współczynników __beta__. W tym celu tworzę _dataframe_ który posłuży mi do dalszej analizy. Każda kolumna owej tabeli odpowiada badanym współczynnikom __beta__, a w wierszach znajdują się informacje o wartościach zwracanych przez _GDA_ dla losowych  punktów startowych.

In [59]:
b_arr = np.arange(0.0001, 0.02, 0.0001)
num_iters = 100
data = performance = {np.round(b, 4):(test_beta(g, 2, grad_g, b, [-10, 10], num_iters)) for b in b_arr}
df = pd.DataFrame(columns=[b for b in performance.keys()])
for col in df.columns:
    df[col] = performance[col]

In [60]:
df

,0.0001,0.0002,0.0003,0.0004,0.0005,0.0006,0.0007,0.0008,0.0009,0.0010,...,0.0190,0.0191,0.0192,0.0193,0.0194,0.0195,0.0196,0.0197,0.0198,0.0199
0,-37.623379,4.112502,-5.421180,-6.164876,-6.447927,-42.219828,-42.353981,NaN,-42.470523,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11.563354,-3.052728,-5.718749,-41.745241,-6.439778,-6.617146,-6.725611,-42.409775,-42.453700,-6.866720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-38.877793,-39.034329,-5.889439,-42.435436,-6.482804,NaN,NaN,-6.785300,NaN,-6.887193,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6.064581,-39.288679,-41.231506,-41.768082,-6.438708,-6.618377,-42.322012,NaN,-6.836166,-42.483372,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-26.167191,-39.082211,-41.058498,-6.128705,-42.071083,-42.528337,-42.323251,-6.790952,NaN,-42.491889,...,-31.990358,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-26.579092,-3.258877,-5.378299,-41.814061,-6.439328,-6.613061,-42.626458,-42.403005,-42.495792,NaN,...,NaN,NaN,NaN,-34.5092,NaN,NaN,NaN,NaN,NaN,NaN
96,-26.149542,-41.623764,-41.054857,-41.749973,-42.065271,-42.293823,-42.333921,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-30.643889,NaN,NaN,NaN,NaN
97,168.148832,-39.097251,-5.404686,-6.123944,-6.441388,-6.775984,-6.749003,-6.785886,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,7.753504,-39.085247,-41.150573,-6.132693,-40.449367,-36.752778,NaN,-42.424246,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
df.describe()

,0.0001,0.0002,0.0003,0.0004,0.0005,0.0006,0.0007,0.0008,0.0009,0.0010,...,0.0190,0.0191,0.0192,0.0193,0.0194,0.0195,0.0196,0.0197,0.0198,0.0199
count,100.000000,100.000000,100.000000,100.000000,100.000000,82.000000,75.000000,68.000000,47.000000,41.000000,...,4.000000,1.000000,2.000000,5.000000,3.000000,3.000000,3.000000,2.000000,2.000000,1.000000
mean,-7.795806,-18.111826,-21.208698,-28.320165,-24.517863,-28.692106,-25.253794,-24.647672,-29.606991,-26.855895,...,-19.498426,-7.006117,-31.772428,-28.934284,-41.026256,-27.622204,-31.338128,-7.006069,-18.382856,-40.033043
std,25.739516,17.894127,17.791744,17.435702,17.854504,17.442555,17.960350,17.942885,17.293170,17.915661,...,14.424666,NaN,0.779232,13.253192,1.996700,2.720999,8.027900,0.000165,16.089280,NaN
min,-40.268957,-42.496251,-42.626227,-42.627258,-42.597363,-42.627678,-42.626458,-42.626308,-42.625260,-42.626492,...,-31.990749,-7.006117,-32.323429,-42.337882,-42.614728,-30.643889,-40.521573,-7.006186,-29.759695,-40.033043
25%,-26.960817,-39.124067,-41.072799,-41.826751,-42.070831,-42.282589,-42.346271,-42.416102,-42.475848,-42.496543,...,-31.990456,-7.006117,-32.047928,-34.509200,-42.146972,-28.750420,-34.180668,-7.006127,-24.071276,-40.033043
50%,-1.959301,-4.126148,-6.702676,-41.745840,-37.560744,-42.231802,-42.322012,-24.704483,-42.454003,-42.483476,...,-19.498339,-7.006117,-31.772428,-32.232632,-41.679217,-26.856951,-27.839764,-7.006069,-18.382856,-40.033043
75%,9.537690,-3.129086,-5.414354,-6.193203,-6.460869,-6.789935,-6.746594,-6.806229,-6.901012,-6.869586,...,-7.006310,-7.006117,-31.496928,-28.585757,-40.232020,-26.111362,-26.746405,-7.006010,-12.694437,-40.033043
max,168.148832,4.112502,2.414743,-2.485636,-0.579384,-0.515201,-3.145826,-6.780883,-6.815795,-6.423692,...,-7.006276,-7.006117,-31.221428,-7.005952,-38.784824,-25.365773,-25.653047,-7.005952,-7.006018,-40.033043


Następnie rysuję wykres wartości funkcji dla punktów zwracanych przez _GDA_, dla odpowiednich wartosci współczynników __beta__. 

In [62]:
layout = go.Layout(width=700, height=500,
                   title_text='Algorithm outputs in function of beta value',
                   plot_bgcolor='DarkSeaGreen',
                   xaxis_title='beta',
                   yaxis_title='min(g(x, y)) from GDA')
fig = go.Figure(layout=layout)
for _, row in df.iterrows():
    fig.add_trace(go.Scatter(mode='markers', marker_color='DarkSlateGrey', 
                             opacity=0.5, x=df.columns, y=row, showlegend = False))
fig.show()

Na tym wykresie nie widać jednak ile iteracji dla każdej wartości __beta__ nie osiągneło minimum. Wobec tego następny wykres przedstawia stosunek ilości nieudanych iteracji do wykonanych.

In [63]:
fail_rate = df.isna().sum()/num_iters

layout = go.Layout(width=700, height=500,
                title_text='Beta fail (out of bounds) rate',
                plot_bgcolor='DarkSeaGreen',
                xaxis_title='beta',
                yaxis_title='failed/total iterations')
fig = go.Figure(data=[go.Bar(x=df.columns, y=fail_rate, marker_color='DarkSlateGrey')], layout=layout)
fig.show()

Następnie po ustaleniu maksymalnego błędu wyznaczam najlepszy parametr __beta__, taki, który daje możliwie najniższe wyniki _GDA_.

In [64]:
max_error = 0.1
tmp_df = df[df.columns[df.isna().sum()/num_iters < max_error]]
best_b = tmp_df.columns[tmp_df.sum().argmin()]
print(best_b)

0.0004


Na koniec rysuję wykres wartości osiaganych przez _GDA_ dla znalezionego parametru __beta__. 

In [65]:
layout = go.Layout(width=700, height=500,
                   title_text='Best beta outputs',
                   plot_bgcolor='DarkSeaGreen',
                   xaxis_title='Iteration number',
                   yaxis_title='min(g(x, y)) from GDA')
fig = go.Figure(data=[go.Bar(y=df[best_b], marker_color='DarkSlateGrey')], layout=layout)
fig.update_yaxes(autorange="reversed")
fig.show()

Z powyższego wykresu można ostatecznie wnioskować, że funkcja ta ma dwa minima lokalne. Dla znalezionego parametru beta powinniśmy wobec tego otrzymać wartość minimalną w okolicy _-42_ bądź _-7_.

In [66]:
p = RNG.uniform(-10, 10, 2)
arr = gradient_descent(p, best_b, grad_g, 0.0001, 1000)[0]
g_min = arr[-1]
print(f"argmin(g) = {g_min}")
print(f"Step amount: {len(arr)-1}")
print(f"g(argmin) = {g(g_min)}")

argmin(g) = [ 1.01255865 -0.17625934]
Step amount: 1000
g(argmin) = -6.99667035509362


Analogiczny proces przeprowadziłem dla funkcji jednej zmiennej f(x):

In [67]:
b_arr = np.arange(0.0001, 0.02, 0.0001)
num_iters = 100
performance = {np.round(b, 4):(test_beta(f, 1, grad_f, b, [-10, 10], num_iters)) for b in b_arr}

df = pd.DataFrame(columns=[b for b in performance.keys()])
for col in df.columns:
    df[col] = performance[col]
max_error = 0.1
tmp_df = df[df.columns[df.isna().sum()/num_iters < max_error]]
best_b = tmp_df.columns[tmp_df.sum().argmin()]
print(best_b)

0.0005


In [68]:
p = RNG.uniform(-10, 10, 1)
arr = gradient_descent(p, best_b, grad_f, 0.0001, 1000)[0]
f_min = arr[-1]
print(f"argmin(f) = {f_min}")
print(f"Step amount: {len(arr)-1}")
print(f"f(argmin) = {f(f_min)}")

argmin(f) = [1.01255986]
Step amount: 302
f(argmin) = -7.006322161188656


Ostatecznie, przy _10000_ iteracji dla funkcji _g(x, y)_ algorytm znalajduje minima: _g([-1.41, 0.05]) = -42.63_ oraz _g([1.01 0.05]) = -7.01_, a także dla funkcji f(x): _f([-1.41]) = -42.63_ i 
 _f([1.01]) = -7.01_, gdzie to pierwsze to minima globalne.

#### Wnioski


Ze względu na pewną ułomność tego algorytmu, oraz naturę badanych funkcji, algorytm ma tendencję wpadać w minima lokalne, a także wystrzeliwywać wyniki w kosmos kiedy pochodne badanych funkcji są duże oraz gwałtownie maleją w okolicy minimum. Najbezpieczniej wobec tego byłoby wybrać możliwie najmniejszy parametr __beta__ i możliwie najwięcej iteracji, co oczywiście nie jest możliwe jeśli zależny nam na czasie. Problem dotyczący wpadania w minima lokalne można próbować rozwiązać stosując stochastyczny algorytm spadku gradientu, który zamiast używać dokładnie obliczonego gradientu, korzysta z aproksymowanego gradientu który posiada przez to losowe zakłócenia. Ze względu na to że algorytm nie porusza się wtedy dokładnie w kierunku spadku wartości funkcji optymalizowanej, istnieje szansa, że lokalne minima zostaną ominięte. 